In [1]:
import cv2
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing import image

In [2]:
# Load the trained model
model = tf.keras.models.load_model('experiment/models/facial_recognition_model.keras')

#vgg_transfer_learning_model
#facial_recognition_model

# Get the class indices from the training generator (assuming you have it saved or remember the mapping)
class_indices = {'Aratan': 0,
 'Flores': 1,
 'Harper': 2,
 'Ilogon': 3,
 'Malaluan': 4,
 'Matilla': 5,
 'Rasonabe': 6,
 'Tamayo': 7}
class_names = list(class_indices.keys())


In [3]:
# Load the pre-trained face detector (Haar Cascade Classifier)
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Define the function to preprocess the face region for model input
def preprocess_frame(face):
    face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB if the model expects RGB input
    face = cv2.resize(face, (224, 224))  # Resize the face region to match the model's input size
    face = np.expand_dims(face, axis=0)  # Expand dimensions to match the batch format
    face = face.astype('float32') / 255.0  # Normalize the image
    return face

# Set the confidence threshold
CONFIDENCE_THRESHOLD = 0.7  # Adjust as needed

# Start capturing video from the webcam
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()

try:
    while True:
        # Capture frame-by-frame
        ret, frame = cap.read()

        if not ret:
            print("Error: Failed to read frame from webcam.")
            break

        # Flip the frame horizontally to mirror the image
        frame = cv2.flip(frame, 1)

        # Convert the frame to grayscale for face detection
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Detect faces in the frame
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

        # Initialize predictions array for displaying all class confidences
        predictions_text = []

        # For each detected face, predict and draw a bounding box with the name
        for (x, y, w, h) in faces:
            # Extract the face region
            face = frame[y:y+h, x:x+w]

            # Preprocess the face for the model
            preprocessed_face = preprocess_frame(face)

            # Make predictions
            predictions = model.predict(preprocessed_face)[0]  # Extract predictions for the batch
            max_confidence = np.max(predictions)  # Get the highest confidence score
            predicted_class_index = np.argmax(predictions)

            # Check if the highest confidence is above the threshold
            if max_confidence >= CONFIDENCE_THRESHOLD:
                predicted_class = class_names[predicted_class_index]
            else:
                predicted_class = "Unknown"

            # Draw a rectangle around the face
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

            # Display the predicted class (name or "Unknown") above the face
            cv2.putText(frame, f'{predicted_class} ({max_confidence:.2f})', (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

            # Prepare confidence text for all classes
            for i, confidence in enumerate(predictions):
                predictions_text.append(f'{class_names[i]}: {confidence:.2f}')

        # Create a blank canvas to display the predictions below the camera feed
        frame_height, frame_width = frame.shape[:2]

        # Adjust canvas height to accommodate 8 classes (20 px per class + some padding)
        canvas_height = 20 * 8 + 20  # 8 classes + padding
        blank_canvas = np.zeros((canvas_height, frame_width, 3), dtype=np.uint8)

        # Add predictions text to the blank canvas
        for i, text in enumerate(predictions_text[:8]):  # Limit to 8 classes
            y_offset = 20 + i * 20
            cv2.putText(blank_canvas, text, (10, y_offset), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 1)

        # Combine the camera frame and the blank canvas
        combined_frame = np.vstack((frame, blank_canvas))

        # Display the resulting frame
        cv2.imshow('Facial Recognition with Confidence Levels', combined_frame)

        # Check for key press and exit on 'q'
        if cv2.waitKey(1) & 0xFF == ord('q'):
            breakQ

finally:
    # Ensure resources are released properly
    cap.release()
    cv2.destroyAllWindows()


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 661ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━

NameError: name 'breakQ' is not defined